In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Loading Data

In [ ]:
df=pd.read_csv('../input/ethusd-cryptocurrency/Binance_ETHUSDT_d - Binance_ETHUSDT_d.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
from datetime import date
import json
import requests
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, LSTM
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error
%matplotlib inline
sns.set_style('darkgrid')
import plotly.graph_objects as go

## Visualisation of Ethereum Prices using its trading features

In [ ]:
fig = go.Figure(data=go.Ohlc(x=df['date'],
                    open=df['open'],
                    high=df['high'],
                    low=df['low'],
                    close=df['close']))
fig.show()

In [ ]:
import plotly.express as px
fig = px.line(df, x='date', y='close')
fig.show()

## Min-Max Scaling data 

In [ ]:
close_data=df.reset_index()['close']

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaled_data=scaler.fit_transform(np.array(close_data).reshape(-1,1))

## Train/Test Splitting Data

In [ ]:
train_size=int(len(scaled_data)*0.7)
test_size=len(scaled_data)-train_size
train_data,test_data=scaled_data[0:train_size,:],scaled_data[train_size:len(scaled_data),:]

In [ ]:
train_size,test_size

In [ ]:
def time_series_data(dataset,step):
    x,y=[],[]
    for i in range(len(dataset)-step-1):
        # creating a series of values
        series=dataset[i:(i+step),0]
        # appending that series to x
        x.append(series)
        # appending the next values of series x after step to y
        y.append(dataset[i+step,0])
    return np.array(x),np.array(y)

In [ ]:
step=9
x_train,y_train=time_series_data(train_data,step)
x_test,y_test=time_series_data(test_data,step)

In [ ]:
x_train.shape,y_train.shape

In [ ]:
x_train =x_train.reshape(x_train.shape[0],x_train.shape[1] , 1)
x_test = x_test.reshape(x_test.shape[0],x_test.shape[1] , 1)

## Model Building

In [ ]:
model=Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(49,1)))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='mean_squared_error',optimizer='adam')

In [ ]:
model.summary()

In [ ]:
history=model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=50,batch_size=64)

## Plotting losses

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(loss))
plt.figure(figsize=(15,4))
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title("Training and Validation Loss")
plt.legend()
plt.show()

## Making Predictions

In [ ]:
train_predict=model.predict(x_train)
test_predict=model.predict(x_test)

In [ ]:
# doing inverse transform to scale it back to original scale
train_predict=scaler.inverse_transform(train_predict)
test_predict=scaler.inverse_transform(test_predict)

In [ ]:
y_test=scaler.inverse_transform(y_test.reshape(y_test.shape[0],1))

## Evaluation

In [ ]:
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(y_test,test_predict))

In [ ]:
plt.figure(figsize=(14,5))
plt.plot(y_test, color = 'red', label = 'Real Ethereum Price')
plt.plot(test_predict, color = 'green', label = 'Predicted Ethereum Price')
plt.title('Ethereum Price Prediction using RNN-LSTM')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend()
plt.show()

### Foreacsting for next 30 days